# Visualize UMAP for CP features

In [1]:
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(dplyr))

In [2]:
# Set directory and file structure
umap_dir <- file.path("results")
umap_files <- list.files(umap_dir, full.names = TRUE)
print(umap_files)

output_fig_dir <- file.path("figures")
umap_prefix <- "UMAP_"
plate_suffix <- "_sc_norm_fs_cellprofiler.csv.gz.tsv.gz"

# Define output figure paths
output_umap_files <- list()
for (umap_file in umap_files) {
    # Manipulate the 
    plate <- stringr::str_remove(
        stringr::str_remove(
            unlist(
                strsplit(umap_file, "/")
            )[2],
            umap_prefix
        ),
        plate_suffix
    )
    output_umap_files[plate] <- file.path(
        output_fig_dir,
        paste0(umap_prefix, plate)
    )
}
        
print(output_umap_files)

[1] "results/UMAP_localhost220512140003_KK22-05-198_sc_norm_fs_cellprofiler.csv.gz.tsv.gz"               
[2] "results/UMAP_localhost220513100001_KK22-05-198_FactinAdjusted_sc_norm_fs_cellprofiler.csv.gz.tsv.gz"
$`localhost220512140003_KK22-05-198`
[1] "figures/UMAP_localhost220512140003_KK22-05-198"

$`localhost220513100001_KK22-05-198_FactinAdjusted`
[1] "figures/UMAP_localhost220513100001_KK22-05-198_FactinAdjusted"



In [3]:
# Load data
umap_cp_df <- list()
for (plate in names(output_umap_files)) {
    # Find the umap file associated with the plate
    umap_file <- umap_files[stringr::str_detect(umap_files, plate)]
    
    # Load in the umap data
    df <- readr::read_tsv(
        umap_file,
        col_types = readr::cols(
            .default = "d",
            "Metadata_WellRow" = "c",
            "Metadata_WellCol" = "c",
            "Metadata_heart_number" = "c",
            "Metadata_treatment" = "c",
            "Metadata_dose" = "c",
            "Metadata_ImageNumber" = "c",
            "Metadata_Plate" = "c",
            "Metadata_Well" = "c",
            "Metadata_Cytoplasm_Parent_Cells" = "c",
            "Metadata_Cytoplasm_Parent_Nuclei" = "c",
            "Metadata_Cells_Number_Object_Number" = "c",
            "Metadata_Nuclei_Number_Object_Number" = "c"
        )
    )
    
    cell_count_df <- df %>%
        dplyr::group_by(Metadata_Well) %>%
        dplyr::count()

    umap_cp_df[[plate]] <- df %>%
        dplyr::left_join(cell_count_df, by = "Metadata_Well")
}

## Fix dose ordering for plots

In [4]:
# Order the dose column low to high
dose_unit = "uM"
for (plate in names(umap_cp_df)) {
    # Pull the associated dataframe
    df <- umap_cp_df[[plate]]
    
    # Extract dose order and sort, then recompile
    dose_order <- paste0(
        sort(
            as.numeric(
                stringr::str_remove(
                    names(
                        table(df$Metadata_dose)
                    ),
                    dose_unit
                )
            )
        ),
        dose_unit
    )

    # Perform the factor ordering
    umap_cp_df[[plate]]$Metadata_dose <- 
        factor(df$Metadata_dose, levels = dose_order)
    }

## Generate UMAPs

In [5]:
for (plate in names(umap_cp_df)) {
    # Treatment UMAP
    output_file <- output_umap_files[[plate]]
    output_file <- paste0(output_file, "_treatment.png")
    
    treatment_gg <- (
        ggplot(umap_cp_df[[plate]], aes(x = UMAP0, y = UMAP1))
        + geom_point(
            aes(color = Metadata_treatment), size = 0.4, alpha = 0.7
        )
        + theme_bw()
        + scale_color_manual(
            name = "Treatment",
            values = c("DMSO" = "#BA5A31", "drug_x" = "#69DC9E")
        )
    )
    
    ggsave(output_file, treatment_gg, dpi = 500, height = 6, width = 6)
    
    # Dose UMAP
    output_file <- output_umap_files[[plate]]
    output_file <- paste0(output_file, "_dose_facet.png")
    
    umap_dose_gg <- (
        ggplot(umap_cp_df[[plate]], aes(x = UMAP0, y = UMAP1))
        + geom_point(
            aes(color = n), size = 0.4, alpha = 0.7
        )
        + theme_bw()
        + facet_wrap("~Metadata_dose")
        + theme(
            strip.background = element_rect(colour = "black", fill = "#fdfff4")
        )
        + scale_color_continuous(name = "Number of\nsingle cells\nper well")
    )
    
    ggsave(output_file, umap_dose_gg, dpi = 500, height = 6, width = 6)
    
}

In [6]:
# Plot with multiple hearts
plate_with_multiple_hearts <- c("localhost220512140003_KK22-05-198")

for (plate in plate_with_multiple_hearts) {
    # Heart UMAP
    output_file <- output_umap_files[[plate]]
    output_file <- paste0(output_file, "_hearts.png")
    
    heart_gg <- (
        ggplot(umap_cp_df[[plate]], aes(x = UMAP0, y = UMAP1))
        + geom_point(
            aes(color = Metadata_heart_number), size = 0.4, alpha = 0.7
        )
        + theme_bw()
    )
    
    ggsave(output_file, heart_gg, dpi = 500, height = 6, width = 6)
    }